# Independent analysis:
## *Testing Theories of American Politics: Elites, Interest Groups, and Average Citizens*
### by: Martin Gilens and Benjamin I. Page
### published in: Perspectives on Politics, 2014, [doi:10.1017/S1537592714001595](https://doi.org/10.1017/S1537592714001595)
##### analysis performed by: Mickey T. Da Silva

tl;dr:
- Paper figures partially replicable despite missing data 
- Those figure parts that can't be replicated are critical to the central argument of economic influence on policy adoption
- Strong policy preference correlation (>= 0.85) among all economic classes
- If a large proportion of a group is in favor of a piece of legislation, it's more likely to be adopted

Caveats:
- Not all data provided (1/3 datasets referenced provided (about 30% of the total dataset))
- Data analysis methods insufficiently explained in the paper. Inability to replicate figures may be due to missing data or unexpected method of analysis.

In [ ]:
import os
from tqdm.auto import tqdm  # Progress bars

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

# Pandas throws some performance warnings (fragmented dataframes). They clutter up the later analyses
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Hard Parameters
FIG_SIZE = (16, 8)
DPI = 300  # Figure dots per inch (resolution)
SAVE_FIGURES = True
SHOW_FIGURES_IN_NOTEBOOK = False # There are many figures in the "Additional Analysis" section
FIGURE_SAVE_DIR = os.path.join("..", "images")  # Redefine as you see fit

In [ ]:
# Utility Functions
def make_return_path(path_in: str) -> str:
    """
    Allows you to simultaneously make a directory tree and define it as a string
    """
    if not os.path.exists(path_in):
        try:
            os.makedirs(path_in)
        except:
            raise Exception(f"Unable to make directories! {path_in}")

    return path_in

In [ ]:
# Calculation Functions
def calc_net_interest_group_favor(df: pd.DataFrame) -> pd.DataFrame:
    """
    Based on pg. 569, col. 2, 
        ln(# Strongly Favor + (0.5 * # Somewhat Favor) + 1) - ln(# Strongly Oppose + (0.5 * # Somewhat Oppose) + 1)
    NOTE: This function doesn't match the above formula because the above formula doesn't reproduce Figure 1c.
    """
    return (
        df["INTGRP_STFAV"]
        + 0.5 * df["INTGRP_SWFAV"]
        - df["INTGRP_STOPP"]
        - 0.5 * df["INTGRP_SWOPP"]
    )

In [ ]:
# Plotting Functions
def create_economic_group_preference_plot(
    raw_data: pd.DataFrame,
    adopted_policy_df: pd.DataFrame,
    total_policies: int,
    df_field: str,
    plot_title: str,
    fig_name: str,
    fig_out_dir: str = FIGURE_SAVE_DIR,
    override_preference_y_lim: bool = False,
) -> None:
    n_bins = 10
    bin_step = 10

    econ_group_percent_favoring = np.linspace(0, bin_step * n_bins, n_bins + 1)
    econ_group_predicted_prob_adoption = []
    econ_group_percent_of_cases = []
    for bin in range(n_bins):
        # Probabilty of adoption
        econ_group_n_adopted = adopted_policy_df[
            (adopted_policy_df[df_field] > bin * bin_step / 100)
            & (adopted_policy_df[df_field] < (bin + 1) * bin_step / 100)
        ].index.__len__()
        econ_group_n_desired = raw_data[
            (raw_data[df_field] > bin * bin_step / 100)
            & (raw_data[df_field] < (bin + 1) * bin_step / 100)
        ].index.__len__()
        if econ_group_n_desired == 0:
            econ_group_predicted_prob_adoption.append(None)
        else:
            econ_group_predicted_prob_adoption.append(
                econ_group_n_adopted / econ_group_n_desired
            )

        # Percent of total legislative cases
        econ_group_percent_of_cases.append(econ_group_n_desired / total_policies * 100)

    _, ax = plt.subplots(figsize=FIG_SIZE)
    ax.plot(
        econ_group_percent_favoring[0:n_bins]
        + bin_step / 2,  # Aligning bars with the expected region
        econ_group_predicted_prob_adoption,
        linewidth=5,
        color="k",
    )
    ax.set_xlim([0, 100])
    if override_preference_y_lim:
        ax.set_ylim([0, 1])
    else:
        ax.set_ylim(
            [0.0, 0.7]
        )  # NOTE: The paper uses % everywhere except here for some reason.
    ax.set_xlabel(
        "Percent favoring proposed policy changes [%]"
    )  # Units added because it's sloppy to do otherwise
    ax.set_ylabel("Predicted probability of adoption [-]")
    ax.set_xticks(econ_group_percent_favoring)
    ax.set_title(plot_title)
    ax.set_zorder(1)  # forces the black line over grey bars
    ax.patch.set_visible(False)  # but don't hide the grey bars

    ax2 = ax.twinx()
    ax2.bar(
        econ_group_percent_favoring[0:n_bins] + bin_step / 2,
        econ_group_percent_of_cases,
        width=bin_step,
        color="grey",
    )
    ax2.set_ylabel("Percent of cases (grey columns) [%]")
    ax2.set_ylim([0, 40])

    if SAVE_FIGURES:
        fig_path = os.path.join(make_return_path(fig_out_dir), fig_name)
        plt.savefig(fig_path, dpi=DPI)

    if SHOW_FIGURES_IN_NOTEBOOK:
        plt.show()
    else:
        plt.close()


def create_correlation_plot(
    df: pd.DataFrame,
    df_cols_to_correlate: list[str],
    human_col_names: list[str] = None,
    fig_suptitle: str = "",
    fig_out_dir: str = "",
    fig_name: str = "",
) -> None:
    if human_col_names is None:
        human_col_names = df_cols_to_correlate

    correlation = df[df_cols_to_correlate].corr(method="pearson")

    # Just keep one half of the correlation matrix; it's duplicated across the primary diagonal
    correlation = np.tril(correlation, k=0)
    correlation = np.where(correlation == 0, np.nan, correlation)

    fig, ax = plt.subplots(figsize=FIG_SIZE, nrows=1, ncols=2)
    fig.suptitle(fig_suptitle)

    im0 = ax[0].matshow(correlation, vmin=0, vmax=1)
    ax[0].set_xticks(range(len(human_col_names)), human_col_names, rotation=45)
    ax[0].set_yticks(range(len(human_col_names)), human_col_names)
    fig.colorbar(im0, ax=ax[0])

    im1 = ax[1].matshow(correlation)
    ax[1].set_xticks(range(len(human_col_names)), human_col_names, rotation=45)
    ax[1].set_yticks(range(len(human_col_names)), human_col_names)
    fig.colorbar(im1, ax=ax[1])

    if SAVE_FIGURES:
        fig_path = os.path.join(
            make_return_path(fig_out_dir),
            fig_name,
        )
        plt.savefig(fig_path, dpi=DPI)
    if SHOW_FIGURES_IN_NOTEBOOK:
        plt.show()
    else:
        plt.close()


def create_tiered_group_plots(
    raw_data: pd.DataFrame,
    adopted_policy_df: pd.DataFrame,
    tiered_prop: str,
    n_intracategory_levels: int = 12
) -> None:
    favor_oppose_dontknow_tags = ["fav", "opp", "dk"]
    raw_total_edu_respondants = [
        (
            raw_data[[f"{tiered_prop}{i + 1}_{favop}" for favop in favor_oppose_dontknow_tags]].sum(
                axis=1
            )
        )
        for i in range(n_intracategory_levels)
    ]  # yes, these are 1-indexed and it hurts

    raw_props_in_favor = [
        (raw_data[f"{tiered_prop}{i + 1}_fav"] / raw_total_edu_respondants[i])
        for i in range(n_intracategory_levels)
    ]

    # Normal check
    _ = plt.figure(figsize=FIG_SIZE)
    for i, prop_in_favor in enumerate(raw_props_in_favor):
        try:
            plt.hist(prop_in_favor, bins=10, alpha=0.5, label=f"Level {i + 1}")
        except:
            pass
    plt.title(f"{tiered_prop.capitalize()} Breakdown - Policy Favorability Check")
    plt.xlabel("Proportion in favor of legislation [-]")
    plt.ylabel("Count of Policies [-]")
    plt.legend()

    if SAVE_FIGURES:
        fig_dir = make_return_path(
            os.path.join(FIGURE_SAVE_DIR, "additional-analysis", f"{tiered_prop}-breakdown")
        )
        fig_path = os.path.join(fig_dir, "normal-distribution-check.png")
        plt.savefig(fig_path, dpi=DPI)

    if SHOW_FIGURES_IN_NOTEBOOK:
        plt.show()
    else:
        plt.close()

    # Policy adoption proportion
    adopted_total_respondants = [
        (
            adopted_policy_df[
                [
                    f"{tiered_prop}{i + 1}_{favop}"
                    for favop in favor_oppose_dontknow_tags
                ]
            ].sum(axis=1)
        )
        for i in range(n_intracategory_levels)
    ]
    adopted_pred = [
        (adopted_policy_df[f"{tiered_prop}{i + 1}_fav"] / adopted_total_respondants[i])
        for i in range(n_intracategory_levels)
    ]
    for i in (pb := tqdm(range(n_intracategory_levels))):
        pb.desc = f"Plotting {tiered_prop} {i}..."
        raw_data[f"pred_{tiered_prop}{i}"] = raw_props_in_favor[i]
        adopted_policy_df[f"pred_{tiered_prop}{i}"] = adopted_pred[i]
        n_policies = raw_props_in_favor[i].index.__len__()
        create_economic_group_preference_plot(
            raw_data,
            adopted_policy_df,
            n_policies,
            f"pred_{tiered_prop}{i}",
            f"{tiered_prop.capitalize()} Group {i} Policy Influence",
            f"{tiered_prop}-group-{i}-influence.png",
            os.path.join(
                FIGURE_SAVE_DIR,
                "additional-analysis",
                f"{tiered_prop}-breakdown",
                "preferences",
            ),
        )

    # Correlation
    cols = [f"pred_{tiered_prop}{i}" for i in range(n_intracategory_levels)]
    create_correlation_plot(
        raw_data,
        cols,
        fig_suptitle=f"Policy adoption preference, correlation between {tiered_prop.capitalize()} brackets\n(Same figure, different color ranges to emphasize similarity)",
        fig_out_dir=os.path.join(
            FIGURE_SAVE_DIR, "additional-analysis", f"{tiered_prop}-breakdown"
        ),
        fig_name=f"{tiered_prop}-group-preference-correlation.png",
    )

In [ ]:
# Setup
if SAVE_FIGURES:
    make_return_path(FIGURE_SAVE_DIR)

    additional_out_dirs = ["additional-analysis"]
    for out_dir in additional_out_dirs:
        make_return_path(os.path.join(FIGURE_SAVE_DIR, out_dir))

raw_data = pd.read_stata(
    os.path.join("..", "data", "raw-data.dta")
)  # Strange filetype. Less efficient than a CSV?
total_policies = raw_data.index.__len__()
adopted_policy_df = raw_data[raw_data["OUTCOME"] != 0]
n_adopted_policies = adopted_policy_df.index.__len__()

## Replicate Figure 1

### Figure 1a: Average Citizens' Preferences
Notes: 
- Average Citizen is defined as the 50th income percentile
- Deviation from paper: Units added to axis labels for clarity and completeness
- 2/3 axes in % but one given as a floating [0, 1] probability in paper. Kept that schema for consistency but strange choice.

In [ ]:
create_economic_group_preference_plot(
    raw_data,
    adopted_policy_df,
    total_policies,
    "pred50_sw",
    "Average Citizens' Preferences",
    "average-citizens-preferences.png",
)

#### Figure 1a Conclusion:
- Percent of cases successfully replicated but predicted probability of adoption isn't. 
- Correction required to replicate figure not outlined in paper or supplemental materials.
- Predicted probability of adoption here shown leads to opposite conclusion as the paper.
- Paper states "the preferences of ordinary citizens tend to be positively correlated with the preferences of economic elites... \[such that they\] are more or less coincidental beneficiaries rather than causes of the victory.". This implies further processing was required to produce the flat line for average citizens' preferences to decouple them from the economic elites or interest groups but nowhere is this stated. An attempt to reverse engineer this analysis is made in the Additional Analysis section of this notebook.

### Figure 1b: Economic Elites' Preferences
Notes: 
- Economic Elite is defined as the 90th income percentile
- Deviation from paper: Units added to axis labels for clarity and completeness
- 2/3 axes in % but one given as a floating [0, 1] probability in paper. Kept that schema for consistency but strange choice.

In [ ]:
create_economic_group_preference_plot(
    raw_data,
    adopted_policy_df,
    total_policies,
    "pred90_sw",
    "Economic Elites' Preferences",
    "economic-elites-preferences.png",
)

#### Figure 1b Conclusion:
- Predicted probability much closer to the paper; it's not perfect but it's pretty close. I could believe the difference is because of the missing 70% of the dataset but it's still confusing why the Percent of cases would match so well but the predicted probability of adoption wouldn't. What are the chances the missing data have the same Percent of cases distribution?
- Percent of cases again replicated.

### Figure 1c: Interest Group Alignments
Notes: 
- Some interest group data missing (27 cases). Omitted from analysis.
- Deviation from paper: Units added to axis labels for clarity and completeness
- Deviation from paper: Left y-axis (predicted probability of adoption) scaled to 1 rather than 0.7
- 2/3 axes in % but one given as a floating [0, 1] probability in paper. Kept that schema for consistency but strange choice.

In [ ]:
# Sorry for how messy this one is, it's similar but not quite the same as the other two because of the "net interest groups" rather than "bins"
policy_adopted_ig_included = adopted_policy_df[
    adopted_policy_df["INTGRP_STFAV"].notna()
]
raw_ig_included = raw_data[raw_data["INTGRP_STFAV"].notna()]


net_interest_groups_favoring_adopted = calc_net_interest_group_favor(
    policy_adopted_ig_included
)
net_interest_groups_favoring_raw = calc_net_interest_group_favor(raw_ig_included)

n_bins = 13
bin_step = 4
bin_range = (-24, 24)  # inclusive
bins = np.linspace(bin_range[0], bin_range[1], n_bins)

interest_groups_predicted_prob_adoption = [
    (
        (net_interest_groups_favoring_adopted > i)
        & (net_interest_groups_favoring_adopted < (i + bin_step))
    ).sum(skipna=True)
    / (
        (net_interest_groups_favoring_raw > i)
        & (net_interest_groups_favoring_raw < (i + bin_step))
    ).sum(skipna=True)
    * 100
    for i in bins
]
interest_groups_percent_of_cases = [
    (
        (net_interest_groups_favoring_raw > i)
        & (net_interest_groups_favoring_raw < (i + bin_step))
    ).sum(skipna=True)
    / raw_ig_included.index.__len__()
    * 100
    for i in bins
]

fig, ax = plt.subplots(figsize=FIG_SIZE)
ax.plot(
    bins + bin_step / 2,  # Align
    interest_groups_predicted_prob_adoption,
    linewidth=5,
    color="k",
)
ax.set_xlim(bin_range)
ax.set_ylim([0.0, 110])
ax.set_xlabel("Net interest groups in support or opposition [#]")
ax.set_ylabel("Predicted probability of adoption [%]")
# ax.set_xticks(net_interest_groups_favoring_adopted)
ax.set_title("Interest Groups' Preferences")
ax.set_zorder(1)
ax.patch.set_visible(False)

ax2 = ax.twinx()
ax2.bar(
    bins + bin_step / 2,
    interest_groups_percent_of_cases,
    width=bin_step - 0.1,
    color="grey",
)
ax2.set_ylabel("Percent of cases (grey columns) [%]")
ax2.set_ylim([0, 40])

if SAVE_FIGURES:
    plt.savefig(
        os.path.join(FIGURE_SAVE_DIR, "interest-group-preferences.png"), dpi=DPI
    )

if SHOW_FIGURES_IN_NOTEBOOK:
    plt.show()
else:
    plt.close()

#### Figure 1c Conclusion
- Very noisy predicted probability of adoption; most likely due to how few cases fall in each any bin beyond +-4. Quite possibly the gravest symptom of having a reduced dataset.
- Due to noise, no strong conclusion can be drawn.

### Figure 1 Conclusion:
- Unable to replicate predicted probability of adoption in figures 1a (average citizen) and 1c (interest groups).
- Interest groups most likely lack data to draw a strong conclusion but average citizen must have additional processing required to replicate paper's figure 1a. This analysis isn't outlined neither in the paper -- mathematically or verbally -- nor is it in the supplemental materials.
- Inability to replicate these figures undermines thesis of paper.

## Additional Analysis

### Economic Group Preference Correlation

In [ ]:
income_percentiles = [10, 30, 50, 70, 90]
cols = [f"pred{val}_sw" for val in income_percentiles]
human_col_names = [f"{val}th %ile" for val in income_percentiles]
create_correlation_plot(
    raw_data,
    cols,
    human_col_names,
    fig_suptitle="Policy adoption preference, correlation between income brackets\n(Same figure, different color ranges to emphasize similarity)",
    fig_out_dir=os.path.join(FIGURE_SAVE_DIR, "additional-analysis"),
    fig_name="economic-group-preference-correlation.png",
)

#### Economic Group Preference Correlation Conclusion:
- There's such a strong correlation between all the economic groups that it seems silly to call them separate.

### Normal Distribution Check for Legislation Adoption Preference

In [ ]:
for col, human_readable_col in zip(cols, human_col_names):
    raw_data[col].plot.hist(bins=10, alpha=0.5, label = human_readable_col)
    plt.vlines([raw_data[col].mean()], ymin = 0, ymax = 300, linestyles = "--", colors = "m")

plt.xlabel("Proportion in favor of legislation [-]")
plt.title("Normal distribution check")
plt.legend()

if SAVE_FIGURES:
    fig_path = os.path.join(FIGURE_SAVE_DIR, "additional-analysis", "proportion-in-favor-of-legislation.png")
    plt.savefig(fig_path, dpi = DPI)

if SHOW_FIGURES_IN_NOTEBOOK:
    plt.show()
else:
    plt.close()

#### Normal Distribution Check Conclusion:
- Approximately normal with a slight bias towards preferring to adopt legislation

### Legisative Topic Breakdown

In [ ]:
for legislative_topic in (pb := tqdm(raw_data["XL_AREA"].unique().tolist())):
    base_out_dir = os.path.join(
        FIGURE_SAVE_DIR, "additional-analysis", "legislative-area", "preference"
    )
    pb.desc = f"Plotting {legislative_topic} to {base_out_dir}..."

    n_policies_on_topic = (raw_data["XL_AREA"] == legislative_topic).sum(skipna=True)
    create_economic_group_preference_plot(
        raw_data[raw_data["XL_AREA"] == legislative_topic],
        adopted_policy_df[adopted_policy_df["XL_AREA"] == legislative_topic],
        total_policies=n_policies_on_topic,
        df_field="pred50_sw",
        plot_title=f"Average Citizens' Preference in {legislative_topic} ({n_policies_on_topic}/{total_policies} policies)",
        fig_name=f"average-citizens-preference-{legislative_topic}.png",
        fig_out_dir=make_return_path(os.path.join(base_out_dir, "average-citizens")),
    )

    create_economic_group_preference_plot(
        raw_data[raw_data["XL_AREA"] == legislative_topic],
        adopted_policy_df[adopted_policy_df["XL_AREA"] == legislative_topic],
        total_policies=n_policies_on_topic,
        df_field="pred90_sw",
        plot_title=f"Economic Elites' Preference in {legislative_topic} ({n_policies_on_topic}/{total_policies} policies)",
        fig_name=f"economic-elites-preference-{legislative_topic}.png",
        fig_out_dir=make_return_path(
            os.path.join(
                base_out_dir,
                "economic-elites",
            )
        ),
        override_preference_y_lim=True,
    )

In [ ]:
for legislative_topic in (pb := tqdm(raw_data["XL_AREA"].unique().tolist())):
    base_out_dir = os.path.join(
        FIGURE_SAVE_DIR, "additional-analysis", "legislative-area", "correlation"
    )
    pb.desc = f"Plotting correlation for {legislative_topic} to {base_out_dir}..."
    n_policies_on_topic = (raw_data["XL_AREA"] == legislative_topic).sum(skipna=True)

    create_correlation_plot(
        raw_data[raw_data["XL_AREA"] == legislative_topic],
        cols,
        human_col_names,
        fig_suptitle=f"Policy adoption preference for {legislative_topic} ({n_policies_on_topic}/{total_policies} policies), correlation between income brackets\n(Same figure, different color ranges to emphasize similarity)",
        fig_out_dir=base_out_dir,
        fig_name=f"economic-group-preference-correlation-{legislative_topic}.png",
    )

#### Legisative Topic Citizen Preference Breakdown Conclusion:
- Some categories of legislation lack data such that no strong conclusion can be made.
- It's interesting that Taxation and Foreign Policy are the most numerous by nearly 2x the next highest. This reinforces the authors' caveat that these data only exist for salient issues. In 1981 - 2002 we weren't as worried about government reform or the budget (really?) and that's reflected in these data.
- Maximum correlation on guns is interesting; poor and rich are aligned there. Minimum correlation is likewise interesting; I wouldn't have expected immigration to be as hot a disagreement as it is now. Huh. That goes through the Clinton presidency though. Hm.

### Tiered Breakdown
Notes:
- There are 12 delineations in income, education, ideology, etc. but the provided metadata says there should just be 7. The extra 5 levels aren't always empty such that they won't be neglected here. Ideology and Party Identity do have 7 levels though.
- It's not clear how these 12 (or 7) income levels break down into the 10th, 30th, 50th, 70th, and 90th income percentiles defined as "pred{level}_sw" which is used to generate the only figures in the paper. For this analysis I'll assume that these are completely separate; the focus of this section is education anyways so the income breakdown isn't important here.
- The higher the tiered bracket, the lower the number of surveys taken. Some data also appear to be completely missing form the higher brackets.
- Because of missing data in the higher brackets across the board, take any and all trend conclusions with a heafty grain of salt.

In [ ]:
# Education, Age, Income (not the same as the pred{percentile}_sw from before for some reason)
for tiered_prop in ["edu", "age", "inc"]:
    create_tiered_group_plots(raw_data, adopted_policy_df, tiered_prop)

# Ideology (conservative -> liberal), Party Identification (republican -> democrat)
for tiered_prop in ["ideo", "pid"]:
    create_tiered_group_plots(raw_data, adopted_policy_df, tiered_prop, n_intracategory_levels=7)

#### Tiered Breakdown Conclusion:
- Unsurprisingly, the highest bracket is the least correlated with the lowest bracket. This is especially pronounced with things like ideology and party identity.
- Income has about as much impact as education (though income and education are most likely themselves correlated of course) and significantly less than party identity
- Age is the most innocuous (highest minimum correlation); granted, this is 1981 - 2002. Now we probably see age as more of a factor? It would be interesting to see if the last few years agree with this 20 year study.
- Overall, the more people in a group who agree with a policy, the more likely it's adopted. It can be quite noisy but there are no completely flat groups as shown in the paper's figure 1a.